In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installation
follow the tutorial: https://colab.research.google.com/drive/1SHa43VuHASLjevzO7y3-wPCxHY18-2H6?usp=sharing

Restart your runtime and rerun the *cell*, if Colab asks for it.

In [2]:
# Install sklearn
!pip install scikit-learn==1.5.2

# Install TabPFN
!pip install tabpfn==2.0

# TabPFN Community installs optional functionalities around the TabPFN model
# These include post-hoc ensembles, interpretability tools, and more
!git clone https://github.com/PriorLabs/tabpfn-extensions
!pip install -e tabpfn-extensions[all]

# Install Baselines
!pip install catboost xgboost

# Install example datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 112.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 

In [ ]:
# Find the installation path of tabpfn so that the colab notebook can locate it
#!find / -type d -name "tabpfn_extensions" 2>/dev/null

In [3]:
# append the installation path of tabpfn we found on the last step
import sys

sys.path.append("/content/tabpfn-extensions/src")

In [4]:
pip install scikit-learn==1.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.8

**Now Restart the Runtime**

**Necessary Imports for the Notebook**

The examples in this notebook require the following imports.
Make sure to run this cell before any other cell.


In order to get the fastest predictions you need to enable GPUs for the notebook

In [1]:
import os

# Setup Imports
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer, load_diabetes, load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Markdown, Latex

# Baseline Imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import torch

from tabpfn import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier, AutoTabPFNRegressor

if not torch.cuda.is_available():
    raise SystemError('GPU device not found. For fast training, please enable GPU. See section above for instructions.')

--------

# **EPICAGE**

## **EPICAGE - Internal Dataset**

### DNAm data feature selection

#### 1.Spearman calculation

In [2]:
# Data Path
from pathlib import Path
import os

PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

# Internal Data Paths
meth_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
clinical_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "raw clinical data.csv"
age_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "age.csv"
fold_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "fivefold_id_split.csv"

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import os

# Load data
methylation_df = pd.read_parquet(meth_path)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)

# Convert fold data from wide to long format
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int) # Record at which fold each sample is tested

In [ ]:
save_dir = PROJECT_ROOT / "Our model" / "EPICAGE" / "Boruta" / "feature selection"

# Align indices
common_ids = methylation_df.index.intersection(age_df.index).intersection(folds.index)
methylation_df = methylation_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
folds = folds.loc[common_ids]

# Compute and save Spearman correlations
for fold in range(1, 6):
    print(f"\n[Fold {fold}] Spearman calculation...")

    # Use all samples not in the current fold as training data
    train_idx = folds[folds != fold].index
    X_train_raw = methylation_df.loc[train_idx]
    y_train = age_series.loc[train_idx]

    # Impute missing values with column means (from training set only)
    X_np = X_train_raw.values.astype(np.float32)
    col_means = np.nanmean(X_np, axis=0)
    X_filled = np.where(np.isnan(X_np), col_means, X_np)
    X_train = pd.DataFrame(X_filled, index=train_idx, columns=X_train_raw.columns)

    # Compute Spearman correlations
    corr_list = []
    for col in X_train.columns:
        r, _ = spearmanr(X_train[col], y_train, nan_policy='omit')
        corr_list.append((col, r))

    spearman_df = pd.DataFrame(corr_list, columns=["CpG_ID", "spearman_corr"])
    spearman_df["abs_corr"] = spearman_df["spearman_corr"].abs()
    spearman_df.sort_values("abs_corr", ascending=False, inplace=True)

    # Save result
    spearman_df.to_csv(f"{save_dir}/spearman_fold{fold}.csv", index=False)
    print(f"Saved: spearman_fold{fold}.csv")

'\n# Align indices\ncommon_ids = methylation_df.index.intersection(age_df.index).intersection(folds.index)\nmethylation_df = methylation_df.loc[common_ids]\nage_series = age_df.loc[common_ids, "age"]\nfolds = folds.loc[common_ids]\n\n# Compute and save Spearman correlations\nfor fold in range(1, 6):\n    print(f"\n[Fold {fold}] Spearman calculation...")\n\n    # Use all samples not in the current fold as training data\n    train_idx = folds[folds != fold].index\n    X_train_raw = methylation_df.loc[train_idx]\n    y_train = age_series.loc[train_idx]\n\n    # Impute missing values with column means (from training set only)\n    X_np = X_train_raw.values.astype(np.float32)\n    col_means = np.nanmean(X_np, axis=0)\n    X_filled = np.where(np.isnan(X_np), col_means, X_np)\n    X_train = pd.DataFrame(X_filled, index=train_idx, columns=X_train_raw.columns)\n\n    # Compute Spearman correlations\n    corr_list = []\n    for col in X_train.columns:\n        r, _ = spearmanr(X_train[col], y_tr

### 2. Boruta calculation

Due to environment issues in Colab, this part is executed on a local machine

------

### **EPICAGE - Base Model**

only clinical data & only methylation data & clinical + methylation data


In [2]:
def convert_object_to_category(df):
    df_converted = df.copy()
    for col in df_converted.select_dtypes(include='object').columns:
        df_converted[col] = df_converted[col].astype(str).astype('category')
    return df_converted

In [3]:
# Data Path
from pathlib import Path
import os

PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

# Internal Data Paths
meth_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
clinical_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "raw clinical data.csv"
age_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "age.csv"
fold_path = PROJECT_ROOT / "data_merge" / "Raw Data"/ "fivefold_id_split.csv"
boruta_dir =  PROJECT_ROOT / "Our model" / "EPICAGE" / "Boruta" / "feature selection"
results_dir =  PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_metrics.csv"
prediction_dir =  PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from tabpfn import TabPFNRegressor
import torch

# Load data
methylation_df = pd.read_parquet(meth_path)
clinical_df = pd.read_csv(clinical_path, index_col=0)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)

# Process folds
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int)

# Align IDs
common_ids = methylation_df.index.intersection(clinical_df.index).intersection(age_df.index).intersection(folds.index)
methylation_df = methylation_df.loc[common_ids]
clinical_df = clinical_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
folds = folds.loc[common_ids]

# Results containers
results = []
all_preds = []

# 5-Fold Loop
for fold in range(1, 6):
    print(f"\n=== Fold {fold} Results ===")

    # Load selected CpGs
    boruta_path = f"{boruta_dir}/borutashap_fold{fold}_features.csv"
    selected_cpgs = pd.read_csv(boruta_path, header=None, skiprows=1).squeeze("columns").tolist()

    # Split data
    train_idx = folds[folds != fold].index
    test_idx = folds[folds == fold].index

    # Subset data
    X_cli_train = clinical_df.loc[train_idx]
    X_cli_test = clinical_df.loc[test_idx]
    X_meth_train = methylation_df.loc[train_idx, selected_cpgs]
    X_meth_test = methylation_df.loc[test_idx, selected_cpgs]
    y_train = age_series.loc[train_idx]
    y_test = age_series.loc[test_idx]

    # Combine clinical + methylation
    X_comb_train = pd.concat([X_cli_train, X_meth_train], axis=1)
    X_comb_test = pd.concat([X_cli_test, X_meth_test], axis=1)

    # TabPFN training function
    def train_and_predict(X_train, y_train, X_test):
        model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
        model.fit(X_train, y_train)
        return model.predict(X_test)

    # convert_object_to_category to avoid error
    X_cli_train = convert_object_to_category(X_cli_train)
    X_cli_test = convert_object_to_category(X_cli_test)
    X_comb_train = convert_object_to_category(X_comb_train)
    X_comb_test = convert_object_to_category(X_comb_test)

    # Train models
    pred_cli = train_and_predict(X_cli_train, y_train, X_cli_test)
    pred_meth = train_and_predict(X_meth_train, y_train, X_meth_test)
    pred_comb = train_and_predict(X_comb_train, y_train, X_comb_test)

    # Calculate metrics
    rmse_cli = root_mean_squared_error(y_test, pred_cli)
    mae_cli = mean_absolute_error(y_test, pred_cli)
    r2_cli = r2_score(y_test, pred_cli)

    rmse_meth = root_mean_squared_error(y_test, pred_meth)
    mae_meth = mean_absolute_error(y_test, pred_meth)
    r2_meth = r2_score(y_test, pred_meth)

    rmse_comb = root_mean_squared_error(y_test, pred_comb)
    mae_comb = mean_absolute_error(y_test, pred_comb)
    r2_comb = r2_score(y_test, pred_comb)

    # Print per-fold metrics
    print(f"Phenotypic Clock Layer1  - Fold {fold}  - RMSE: {rmse_cli:.2f}, MAE: {mae_cli:.2f}, R²: {r2_cli:.3f}")
    print(f"Epigenetic Clock Layer1  - Fold {fold}  - RMSE: {rmse_meth:.2f}, MAE: {mae_meth:.2f}, R²: {r2_meth:.3f}")
    print(f"Fusion Clock Layer1      - Fold {fold}  - RMSE: {rmse_comb:.2f}, MAE: {mae_comb:.2f}, R²: {r2_comb:.3f}")

    # Store metrics
    results.append({
        'fold': fold,
        'rmse_cli': rmse_cli,
        'mae_cli': mae_cli,
        'r2_cli': r2_cli,
        'rmse_meth': rmse_meth,
        'mae_meth': mae_meth,
        'r2_meth': r2_meth,
        'rmse_comb': rmse_comb,
        'mae_comb': mae_comb,
        'r2_comb': r2_comb,
    })

    # Store predictions
    all_preds.append(pd.DataFrame({
        'id': test_idx,
        'fold': fold,
        'y_true': y_test.values,
        'pred_cli': pred_cli,
        'pred_meth': pred_meth,
        'pred_comb': pred_comb
    }).set_index('id'))

# Save results
results_df = pd.DataFrame(results)
preds_df = pd.concat(all_preds)


results_df.to_csv(results_dir, index=False)
preds_df.to_csv(prediction_dir)

# Summary
mean_std_result = {}
for metric in ['rmse_cli', 'mae_cli', 'r2_cli', 'rmse_meth', 'mae_meth', 'r2_meth', 'rmse_comb', 'mae_comb', 'r2_comb']:
    mean_val = results_df[metric].mean()
    std_val = results_df[metric].std()
    mean_std_result[metric] = f"{mean_val:.2f} ± {std_val:.2f}"

# Print summary
print("\n=== Overall EPICAGE Clock Layer1 Results ===")
print("Phenotypic Clock Layer1:")
print("  RMSE:", mean_std_result['rmse_cli'])
print("  MAE: ", mean_std_result['mae_cli'])
print("  R²:  ", mean_std_result['r2_cli'])

print("\nEpigenetic Clock Layer1:")
print("  RMSE:", mean_std_result['rmse_meth'])
print("  MAE: ", mean_std_result['mae_meth'])
print("  R²:  ", mean_std_result['r2_meth'])

print("\nFusion Clock Layer1:")
print("  RMSE:", mean_std_result['rmse_comb'])
print("  MAE: ", mean_std_result['mae_comb'])
print("  R²:  ", mean_std_result['r2_comb'])


=== Fold 1 Results ===


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Phenotypic Clock Layer1  - Fold 1  - RMSE: 11.93, MAE: 9.54, R²: 0.208
Epigenetic Clock Layer1  - Fold 1  - RMSE: 8.76, MAE: 6.94, R²: 0.573
Fusion Clock Layer1      - Fold 1  - RMSE: 8.68, MAE: 6.82, R²: 0.581

=== Fold 2 Results ===
Phenotypic Clock Layer1  - Fold 2  - RMSE: 12.50, MAE: 10.06, R²: 0.143
Epigenetic Clock Layer1  - Fold 2  - RMSE: 8.57, MAE: 6.91, R²: 0.597
Fusion Clock Layer1      - Fold 2  - RMSE: 8.52, MAE: 6.85, R²: 0.602

=== Fold 3 Results ===
Phenotypic Clock Layer1  - Fold 3  - RMSE: 12.63, MAE: 10.05, R²: 0.157
Epigenetic Clock Layer1  - Fold 3  - RMSE: 8.68, MAE: 6.85, R²: 0.602
Fusion Clock Layer1      - Fold 3  - RMSE: 8.60, MAE: 6.76, R²: 0.609

=== Fold 4 Results ===
Phenotypic Clock Layer1  - Fold 4  - RMSE: 11.75, MAE: 9.52, R²: 0.213
Epigenetic Clock Layer1  - Fold 4  - RMSE: 8.40, MAE: 6.67, R²: 0.598
Fusion Clock Layer1      - Fold 4  - RMSE: 8.34, MAE: 6.60, R²: 0.603

=== Fold 5 Results ===
Phenotypic Clock Layer1  - Fold 5  - RMSE: 12.46, MAE: 9.9

### **EPICAGE - Ensemble Model**

In [5]:
def convert_object_to_category(df):
    df_converted = df.copy()
    for col in df_converted.select_dtypes(include='object').columns:
        df_converted[col] = df_converted[col].astype(str).astype('category')
    return df_converted

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm
import torch
from tabpfn import TabPFNRegressor
import warnings

warnings.filterwarnings("ignore")

# Paths
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")
meth_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
clinical_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
fold_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "fivefold_id_split.csv"
base_prediction_dir = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"
# Define output path for saving predictions
prediction_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "internal_pca_tabpfn_feature_injection_predictions.csv"

# Load Data
methylation_df = pd.read_parquet(meth_path)
clinical_df = pd.read_csv(clinical_path, index_col=0)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)
preds_df = pd.read_csv(base_prediction_dir, index_col=0)

# Process Fold Index
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int)

# Align IDs
common_ids = (
    methylation_df.index
    .intersection(clinical_df.index)
    .intersection(age_df.index)
    .intersection(folds.index)
)
methylation_df = methylation_df.loc[common_ids]
clinical_df = clinical_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
folds = folds.loc[common_ids]
preds_df = preds_df.loc[common_ids]

# Final Containers
fold_results = []

# Outer CV
rmse_list = []
mae_list = []
r2_list = []

for fold in tqdm(range(1, 6), desc="Outer CV Folds"):
    print(f"\n=== Fold {fold} Results ===")

    test_idx = folds[folds == fold].index
    train_idx = folds[folds != fold].index

    # --- Step 1: Prepare methylation matrices ---
    meth_train_df = methylation_df.loc[train_idx].copy()
    meth_test_df = methylation_df.loc[test_idx].copy()

    # Fill NaNs using training set column means
    meth_train_np = meth_train_df.values
    meth_test_np = meth_test_df.values
    mean_values = np.nanmean(meth_train_np, axis=0)
    meth_train_np = np.where(np.isnan(meth_train_np), mean_values, meth_train_np)
    meth_test_np = np.where(np.isnan(meth_test_np), mean_values, meth_test_np)

    # Standardize
    scaler = StandardScaler()
    meth_train_scaled = scaler.fit_transform(meth_train_np)
    meth_test_scaled = scaler.transform(meth_test_np)

    # PCA
    pca = PCA(n_components=400, random_state=42)
    meth_train_pca = pd.DataFrame(pca.fit_transform(meth_train_scaled),
                                   index=train_idx,
                                   columns=[f'PCA_{i+1}' for i in range(400)])
    meth_test_pca = pd.DataFrame(pca.transform(meth_test_scaled),
                                  index=test_idx,
                                  columns=meth_train_pca.columns)

    # --- Step 2: Prepare full input matrices ---
    X_cli_train = clinical_df.loc[train_idx]
    X_pred_train = preds_df.loc[train_idx, ['pred_cli', 'pred_meth', 'pred_comb']]
    X_train = pd.concat([X_cli_train, X_pred_train, meth_train_pca], axis=1)
    y_train = age_series.loc[train_idx]

    X_cli_test = clinical_df.loc[test_idx]
    X_pred_test = preds_df.loc[test_idx, ['pred_cli', 'pred_meth', 'pred_comb']]
    X_test = pd.concat([X_cli_test, X_pred_test, meth_test_pca], axis=1)
    y_test = age_series.loc[test_idx]

    # Align columns
    X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

    # convert object columns to category to avoid error
    X_train = convert_object_to_category(X_train)
    X_test = convert_object_to_category(X_test)

    # Train & Predict
    model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # save result per fold
    fold_df = pd.DataFrame({
        "id": y_test.index,
        "y_true": y_test.values,
       "y_pred": y_pred,
        "fold": fold
    }).set_index("id")
    fold_results.append(fold_df)

    # Metrics
    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)

    print(f"\nFold {fold} RMSE: {rmse:.2f}")
    print(f"Fold {fold} MAE:  {mae:.2f}")
    print(f"Fold {fold} R²:   {r2:.3f}")

oof_df = pd.concat(fold_results)
oof_df.to_csv(prediction_path)

print("\n=== Overall EPICAGE Fusion Clock Layer2 Results ===")
print(f"RMSE: {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
print(f"MAE:  {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
print(f"R²:   {np.mean(r2_list):.3f} ± {np.std(r2_list):.3f}")

Outer CV Folds:   0%|          | 0/5 [00:00<?, ?it/s]


=== Fold 1 Results ===


Outer CV Folds:  20%|██        | 1/5 [02:14<08:58, 134.56s/it]


Fold 1 RMSE: 8.38
Fold 1 MAE:  6.56
Fold 1 R²:   0.610

=== Fold 2 Results ===


Outer CV Folds:  40%|████      | 2/5 [04:18<06:25, 128.53s/it]

...


Fold 3 RMSE: 8.33
Fold 3 MAE:  6.61
Fold 3 R²:   0.634

=== Fold 4 Results ===


Outer CV Folds:  80%|████████  | 4/5 [08:27<02:05, 125.72s/it]


Fold 4 RMSE: 7.90
Fold 4 MAE:  6.25
Fold 4 R²:   0.644

=== Fold 5 Results ===


Outer CV Folds: 100%|██████████| 5/5 [10:28<00:00, 125.64s/it]


Fold 5 RMSE: 8.11
Fold 5 MAE:  6.38
Fold 5 R²:   0.666

=== Overall EPICAGE Fusion Clock Layer2 Results ===
RMSE: 8.16 ± 0.18
MAE:  6.46 ± 0.13
R²:   0.639 ± 0.018


# Ablation Study

# EPICAGE - no feature injection

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm
import torch
from tabpfn import TabPFNRegressor
import warnings

warnings.filterwarnings("ignore")

# ========== Paths ==========
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")
#meth_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
#clinical_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
fold_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "fivefold_id_split.csv"
prediction_dir = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"

# ========== Load Data ==========
#methylation_df = pd.read_parquet(meth_path)
#clinical_df = pd.read_csv(clinical_path, index_col=0)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)
preds_df = pd.read_csv(prediction_dir, index_col=0)

# ========== Process Fold Index ==========
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int)

# ========== Align IDs ==========
common_ids = (
    methylation_df.index
    .intersection(clinical_df.index)
    .intersection(age_df.index)
    .intersection(folds.index)
)
#methylation_df = methylation_df.loc[common_ids]
#clinical_df = clinical_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
folds = folds.loc[common_ids]
preds_df = preds_df.loc[common_ids]

# ========== Final Containers ==========
all_oof_preds = []
all_y_true = []

# ========== Outer CV ==========
rmse_list = []
mae_list = []
r2_list = []

for fold in tqdm(range(1, 6), desc="Outer CV Folds"):
    print(f"\n================== Fold {fold} ==================")

    test_idx = folds[folds == fold].index
    train_idx = folds[folds != fold].index
    '''
    # ---- Standardize and PCA on Methylation ----
    meth_train = methylation_df.loc[train_idx]
    meth_test = methylation_df.loc[test_idx]

    scaler = StandardScaler()
    meth_train_scaled = scaler.fit_transform(meth_train)
    meth_test_scaled = scaler.transform(meth_test)

    # Fill NaNs
    mean_values = np.nanmean(meth_train_scaled, axis=0)
    meth_train_scaled = np.where(np.isnan(meth_train_scaled), mean_values, meth_train_scaled)
    meth_test_scaled = np.where(np.isnan(meth_test_scaled), mean_values, meth_test_scaled)

    pca = PCA(n_components=400, random_state=42)
    meth_train_pca = pca.fit_transform(meth_train_scaled)
    meth_test_pca = pca.transform(meth_test_scaled)
    '''

    # ---- Combine clinical, OOF preds, PCA ----
    #X_cli_train = pd.get_dummies(clinical_df.loc[train_idx], drop_first=True)
    X_pred_train = preds_df.loc[train_idx, ['pred_cli', 'pred_meth', 'pred_comb']]
    X_train = pd.concat([
        #X_cli_train.reset_index(drop=True),
        X_pred_train.reset_index(drop=True)#,
        #pd.DataFrame(meth_train_pca, columns=[f'PCA_{i+1}' for i in range(400)])
    ], axis=1).astype(np.float32)
    y_train = age_series.loc[train_idx]

    #X_cli_test = pd.get_dummies(clinical_df.loc[test_idx], drop_first=True)
    X_pred_test = preds_df.loc[test_idx, ['pred_cli', 'pred_meth', 'pred_comb']]
    X_test = pd.concat([
        #X_cli_test.reset_index(drop=True),
        X_pred_test.reset_index(drop=True)#,
        #pd.DataFrame(meth_test_pca, columns=[f'PCA_{i+1}' for i in range(400)])
    ], axis=1).astype(np.float32)
    y_test = age_series.loc[test_idx]

    # ---- Align Columns ----
    X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

    # ---- Fit TabPFN ----
    model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
    model.fit(X_train.values, y_train.values)
    y_pred = model.predict(X_test.values)

    all_oof_preds.extend(y_pred)
    all_y_true.extend(y_test)

    # ---- Evaluation ----
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)

    print(f"\nFold {fold} RMSE: {rmse:.2f}")
    print(f"Fold {fold} MAE:  {mae:.2f}")
    print(f"Fold {fold} R²:   {r2:.3f}")

# ========== Final Evaluation ==========
final_rmse = root_mean_squared_error(all_y_true, all_oof_preds)
final_mae = mean_absolute_error(all_y_true, all_oof_preds)
final_r2 = r2_score(all_y_true, all_oof_preds)

print("\n=== Overall EPICAGE Results (No Feature Injection) ===")
print(f"RMSE: {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
print(f"MAE:  {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
print(f"R²:   {np.mean(r2_list):.3f} ± {np.std(r2_list):.3f}")

Outer CV Folds:   0%|          | 0/5 [00:00<?, ?it/s]


================== Fold 1 ==================


Outer CV Folds:  20%|██        | 1/5 [00:00<00:03,  1.05it/s]


Fold 1 RMSE: 8.52
Fold 1 MAE:  6.72
Fold 1 R²:   0.597

================== Fold 2 ==================


Outer CV Folds:  40%|████      | 2/5 [00:01<00:02,  1.08it/s]

...


Fold 3 RMSE: 8.70
Fold 3 MAE:  6.86
Fold 3 R²:   0.600

================== Fold 4 ==================


Outer CV Folds:  80%|████████  | 4/5 [00:03<00:00,  1.07it/s]


Fold 4 RMSE: 8.20
Fold 4 MAE:  6.48
Fold 4 R²:   0.616

================== Fold 5 ==================


Outer CV Folds: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


Fold 5 RMSE: 8.18
Fold 5 MAE:  6.38
Fold 5 R²:   0.660

=== Overall EPICAGE Results (No Feature Injection) ===
RMSE: 8.40 ± 0.20
MAE:  6.64 ± 0.18
R²:   0.617 ± 0.023


# EPICAGE  - Feature injection with feature selection (Sperman + Boruta) rather than PCA

In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from tqdm import tqdm
import torch
from tabpfn import TabPFNRegressor
import warnings

warnings.filterwarnings("ignore")

# Paths
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")
meth_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
clinical_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
fold_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "fivefold_id_split.csv"
prediction_dir = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"
boruta_dir = PROJECT_ROOT / "Our model" / "EPICAGE" / "Boruta" / "feature selection"

# Load data
methylation_df = pd.read_parquet(meth_path)
clinical_df = pd.read_csv(clinical_path, index_col=0)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
folds_df = pd.read_csv(fold_path, index_col=0)
preds_df = pd.read_csv(prediction_dir, index_col=0)

# Process fold indices
folds_long = folds_df.reset_index().melt(id_vars='id', var_name='fold_col', value_name='split')
folds_long['fold'] = folds_long['fold_col'].str.extract(r'splitfold(\d)').astype(int)
folds_long = folds_long[folds_long['split'] == 'test'].drop(columns=['fold_col', 'split'])
folds = folds_long.set_index('id')['fold'].astype(int)

# Align all IDs
common_ids = (
    methylation_df.index
    .intersection(clinical_df.index)
    .intersection(age_df.index)
    .intersection(folds.index)
    .intersection(preds_df.index)
)

methylation_df = methylation_df.loc[common_ids]
clinical_df = clinical_df.loc[common_ids]
age_series = age_df.loc[common_ids]
folds = folds.loc[common_ids]
preds_df = preds_df.loc[common_ids]

# Initialize containers
all_oof_preds = []
all_y_true = []
rmse_list = []
mae_list = []
r2_list = []

# Loop over folds
for fold in tqdm(range(1, 6), desc="Fusion Clock Layer2 (Sperman + Boruta Features)"):
    print(f"\nFold {fold}")

    test_idx = folds[folds == fold].index
    train_idx = folds[folds != fold].index

    # Load Boruta features selected on training fold only
    boruta_path = boruta_dir / f"borutashap_fold{fold}_features.csv"
    selected_cpgs = pd.read_csv(boruta_path, header=None, skiprows=1).squeeze("columns").tolist()

    # Ensure all selected CpGs exist
    for cpg in selected_cpgs:
        if cpg not in methylation_df.columns:
            methylation_df[cpg] = np.nan

    # Prepare methylation data
    X_meth_train = methylation_df.loc[train_idx, selected_cpgs].copy()
    X_meth_test = methylation_df.loc[test_idx, selected_cpgs].copy()

    # Prepare clinical and prediction data
    X_cli_train = clinical_df.loc[train_idx].copy()
    X_cli_test = clinical_df.loc[test_idx].copy()

    X_pred_train = preds_df.loc[train_idx, ['pred_cli', 'pred_meth', 'pred_comb']]
    X_pred_test = preds_df.loc[test_idx, ['pred_cli', 'pred_meth', 'pred_comb']]

    # Combine features
    X_train = pd.concat([X_cli_train.reset_index(drop=True),
                         X_pred_train.reset_index(drop=True),
                         X_meth_train.reset_index(drop=True)], axis=1)

    X_test = pd.concat([X_cli_test.reset_index(drop=True),
                        X_pred_test.reset_index(drop=True),
                        X_meth_test.reset_index(drop=True)], axis=1)

    y_train = age_series.loc[train_idx]
    y_test = age_series.loc[test_idx]

    # Align feature columns
    X_train, X_test = X_train.align(X_test, join="left", axis=1)

    # convert object columns to category to avoid error
    X_train = convert_object_to_category(X_train)
    X_test = convert_object_to_category(X_test)

    # Train and predict
    model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
    model.fit(X_train.values, y_train.values)
    y_pred = model.predict(X_test.values)

    all_oof_preds.extend(y_pred)
    all_y_true.extend(y_test)

    # Evaluate
    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)

    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE : {mae:.2f}")
    print(f"  R²  : {r2:.3f}")

# Final evaluation
print("\nOverall EPICAGE Fusion Clock Layer2 Results (Sperman + Boruta):")
print(f"RMSE: {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
print(f"MAE : {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
print(f"R²  : {np.mean(r2_list):.3f} ± {np.std(r2_list):.3f}")


Fusion Clock Layer2 (Sperman + Boruta Features):   0%|          | 0/5 [00:00<?, ?it/s]


Fold 1


Fusion Clock Layer2 (Sperman + Boruta Features):  20%|██        | 1/5 [00:40<02:40, 40.24s/it]

  RMSE: 8.71
  MAE : 6.84
  R²  : 0.578

Fold 2


Fusion Clock Layer2 (Sperman + Boruta Features):  40%|████      | 2/5 [01:20<02:00, 40.20s/it]

...

  RMSE: 8.72
  MAE : 6.89
  R²  : 0.599

Fold 4


Fusion Clock Layer2 (Sperman + Boruta Features):  80%|████████  | 4/5 [02:31<00:37, 37.28s/it]

  RMSE: 8.50
  MAE : 6.68
  R²  : 0.588

Fold 5


Fusion Clock Layer2 (Sperman + Boruta Features): 100%|██████████| 5/5 [03:11<00:00, 38.29s/it]

  RMSE: 8.48
  MAE : 6.68
  R²  : 0.635

Overall EPICAGE Fusion Clock Layer2 Results (Sperman + Boruta):
RMSE: 8.61 ± 0.10
MAE : 6.82 ± 0.12
R²  : 0.598 ± 0.020


# **EPICAGE - External Evaluation**

- Feature selection is performed using **internal** data only.  
- Three base models are trained on **internal** data:
  - Clinical model
  - Methylation model
  - Combined model
- An ensemble (stacking) model is trained using base model predictions and clinical features from **internal** data.  
- Final performance of all models, including the ensemble, is evaluated on **external** data.

### DNAm data feature selection
#### 1.Spearman calculation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os

# Data path
meth_path = "/content/drive/My Drive/Finalize Data/Raw Data/deduplicated_methylation_data.parquet"
age_path = "/content/drive/My Drive/Finalize Data/Raw Data/age.csv"
clinical_path = "/content/drive/My Drive/Finalize Data/Raw Data/raw clinical data.csv"

# Load data
methylation_df = pd.read_parquet(meth_path)
age_df = pd.read_csv(age_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
clinical_df = pd.read_csv(clinical_path, index_col=0)

# Align indices
common_ids = methylation_df.index.intersection(age_df.index).intersection(clinical_df.index)
methylation_df = methylation_df.loc[common_ids]
age_series = age_df.loc[common_ids, "age"]
clinical_df = clinical_df.loc[common_ids]

print(f"Methylation shape: {methylation_df.shape}")
print(f"Clinical shape: {clinical_df.shape}")
print(f"Age shape: {age_series.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Methylation shape: (4203, 298597)
Clinical shape: (4203, 7)
Age shape: (4203,)


In [2]:
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings("ignore")

# === Spearman correlation ===
print("Step 1: Calculating Spearman correlations...")

X_np = methylation_df.values.astype(np.float32)
col_means = np.nanmean(X_np, axis=0)
X_filled = np.where(np.isnan(X_np), col_means, X_np)
X_df = pd.DataFrame(X_filled, index=methylation_df.index, columns=methylation_df.columns)

corr_list = []
for col in X_df.columns:
    r, _ = spearmanr(X_df[col], age_series, nan_policy='omit')
    corr_list.append((col, r))

spearman_df = pd.DataFrame(corr_list, columns=["CpG_ID", "spearman_corr"])
spearman_df["abs_corr"] = spearman_df["spearman_corr"].abs()
spearman_df.sort_values("abs_corr", ascending=False, inplace=True)
top_cpgs = spearman_df.head(2000)["CpG_ID"].tolist()
print(f"Top 2000 CpGs selected based on Spearman correlation.")

# === save top2000 cpg ===
spearman_path = "/content/drive/My Drive/Finalize Data/feature selection/spearman_all_internal_data.csv"
spearman_df.head(2000).to_csv(spearman_path, index=False)
print(f"Saved top 2000 CpGs to: {spearman_path}")

Step 1: Calculating Spearman correlations...
Top 2000 CpGs selected based on Spearman correlation.
Saved top 2000 CpGs to: /content/drive/My Drive/Finalize Data/feature selection/spearman_all_internal_data.csv


### 2. Boruta calculation

Due to environment issues in Colab, this part is executed on a local machine

----------

### **EPICAGE - Base Model**

only clinical data & only methylation data & clinical + methylation data


In [2]:
def convert_object_to_category(df):
    df_converted = df.copy()
    for col in df_converted.select_dtypes(include='object').columns:
        df_converted[col] = df_converted[col].astype(str).astype('category')
    return df_converted

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import torch
from tabpfn import TabPFNRegressor

# ========== Paths ==========
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

# Internal data
meth_int_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
age_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
cli_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
boruta_path =  PROJECT_ROOT / "Our model" / "EPICAGE" / "Boruta" / "feature selection" / "borutashap_all_internal_features.csv"

# External data
meth_ext_path = PROJECT_ROOT / "data_preprocessing" / "outside_deduplicated_methylation_data.parquet"
age_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_age.csv"
cli_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_raw clinical data.csv"
pred_ext_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_tabpfn_boruta_cli_meth_comb_predictions.csv"

# ========== Load Internal Data ==========
meth_int = pd.read_parquet(meth_int_path)
age_int = pd.read_csv(age_int_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
cli_int = pd.read_csv(cli_int_path, index_col=0)

# Align internal indices
common_int_ids = meth_int.index.intersection(age_int.index).intersection(cli_int.index)
meth_int = meth_int.loc[common_int_ids]
cli_int = cli_int.loc[common_int_ids]
age_int = age_int.loc[common_int_ids, "age"]

# ========== Load Boruta-selected CpGs ==========
selected_cpgs = pd.read_csv(boruta_path, header=None, skiprows=1).squeeze("columns").tolist()

# ========== Train Base Models ==========
from tabpfn import TabPFNRegressor

X_cli_int = cli_int.copy()
X_meth_int = meth_int[selected_cpgs].copy()
X_comb_int = pd.concat([X_cli_int, X_meth_int], axis=1)
y_int = age_int.copy()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_cli = TabPFNRegressor(device=device)
model_meth = TabPFNRegressor(device=device)
model_comb = TabPFNRegressor(device=device)

# convert object columns to category to avoid error
X_cli_int = convert_object_to_category(X_cli_int)
X_comb_int = convert_object_to_category(X_comb_int)

print("Fitting Phenotypic Clock Layer 1...")
model_cli.fit(X_cli_int, y_int)
print("Fitting Epigenetic Clock Layer 1...")
model_meth.fit(X_meth_int, y_int)
print("Fitting Fusion Clock Layer 1...")
model_comb.fit(X_comb_int, y_int)
print("All three models trained.")

Fitting Phenotypic Clock Layer 1...
Fitting Epigenetic Clock Layer 1...
Fitting Fusion Clock Layer 1...
All three models trained.


#### Evaluate base models using external data

In [8]:
# ========== Load External Data ==========
meth_ext = pd.read_parquet(meth_ext_path)
age_ext = pd.read_csv(age_ext_path).rename(columns={"attrib_name": "id", "years_to_birth": "age"}).set_index("id")
cli_ext = pd.read_csv(cli_ext_path, index_col=0)

# Align external indices
common_ext_ids = meth_ext.index.intersection(age_ext.index).intersection(cli_ext.index)
meth_ext = meth_ext.loc[common_ext_ids]
cli_ext = cli_ext.loc[common_ext_ids]
age_ext = age_ext.loc[common_ext_ids, "age"]

# ========== Ensure CpG Feature Consistency ==========
# Compute column-wise means from internal training data
col_means_int = meth_int[selected_cpgs].mean()
meth_ext_fixed = meth_ext.copy()

# Add missing CpG columns using internal mean values
for cpg in selected_cpgs:
    if cpg not in meth_ext_fixed.columns:
        meth_ext_fixed[cpg] = col_means_int[cpg]  # Fill entire column with internal mean

# Reorder columns to match training feature order
meth_ext_fixed = meth_ext_fixed[selected_cpgs]

# Fill NaNs in external CpGs using internal training means
for cpg in selected_cpgs:
    meth_ext_fixed[cpg] = meth_ext_fixed[cpg].fillna(col_means_int[cpg])

X_comb_ext = pd.concat([cli_ext, meth_ext_fixed], axis=1)

# convert object columns to category to avoid error
cli_ext = convert_object_to_category(cli_ext)
X_comb_ext = convert_object_to_category(X_comb_ext)

# ========== Predict ==========
pred_cli = model_cli.predict(cli_ext)
pred_meth = model_meth.predict(meth_ext_fixed)
pred_comb = model_comb.predict(X_comb_ext)

external_preds = pd.DataFrame({
    "id": meth_ext.index,
    "y_true": age_ext.values,
    "pred_cli": pred_cli,
    "pred_meth": pred_meth,
    "pred_comb": pred_comb
}).set_index("id")

# ========== Evaluate ==========
def evaluate_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name:<12} | MAE: {mae:.3f} | RMSE: {rmse:.3f} | R²: {r2:.3f}")

print("\n=== Overall EPICAGE Clock Layer1 Results (External Evaluation) ===")
evaluate_model("Phenotypic Clock Layer 1", external_preds["y_true"], external_preds["pred_cli"])
evaluate_model("Epigenetic Clock Layer 1", external_preds["y_true"], external_preds["pred_meth"])
evaluate_model("Fusion Clock Layer 1", external_preds["y_true"], external_preds["pred_comb"])

# ========== Save ==========
external_preds.to_csv(pred_ext_path)


=== Overall EPICAGE Clock Layer1 Results (External Evaluation) ===
Phenotypic Clock Layer 1 | MAE: 13.098 | RMSE: 16.653 | R²: 0.025
Epigenetic Clock Layer 1 | MAE: 7.343 | RMSE: 9.435 | R²: 0.687
Fusion Clock Layer 1 | MAE: 7.430 | RMSE: 9.496 | R²: 0.683


### **EPICAGE - Ensemble**
- An ensemble (stacking) model is trained using base model predictions and clinical features from **internal** data.
- Use **external** data to evaluate model

In [3]:
from tabpfn import TabPFNRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
from sklearn.decomposition import PCA
import torch

# ========== Paths ==========
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

# Data paths
pred_int_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"
pred_ext_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_tabpfn_boruta_cli_meth_comb_predictions.csv"
meth_int_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
cli_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
meth_ext_path = PROJECT_ROOT / "data_preprocessing" / "outside_deduplicated_methylation_data.parquet"
cli_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_raw clinical data.csv"
age_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_age.csv"
output_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_pca_tabpfn_feature_injection_predictions.csv"

# Load data
meth_int = pd.read_parquet(meth_int_path)
cli_int = pd.read_csv(cli_int_path, index_col=0)
age_int = pd.read_csv(age_int_path).set_index("attrib_name")["years_to_birth"]
meth_ext = pd.read_parquet(meth_ext_path)
cli_ext = pd.read_csv(cli_ext_path, index_col=0)
age_ext = pd.read_csv(age_ext_path).set_index("attrib_name")["years_to_birth"]
pred_int_df = pd.read_csv(pred_int_path, index_col=0)
pred_ext_df = pd.read_csv(pred_ext_path, index_col=0)

# Align indices
common_int_ids = meth_int.index.intersection(cli_int.index).intersection(age_int.index).intersection(pred_int_df.index)
meth_int = meth_int.loc[common_int_ids]
cli_int = cli_int.loc[common_int_ids]
age_int = age_int.loc[common_int_ids]
pred_int_df = pred_int_df.loc[common_int_ids]

common_ext_ids = meth_ext.index.intersection(cli_ext.index).intersection(age_ext.index).intersection(pred_ext_df.index)
meth_ext = meth_ext.loc[common_ext_ids]
cli_ext = cli_ext.loc[common_ext_ids]
age_ext = age_ext.loc[common_ext_ids]
pred_ext_df = pred_ext_df.loc[common_ext_ids]


# ========== Handle Missing Values in Methylation Data ==========
# Ensure external CpG features match internal (reorder & fill missing)
meth_ext = meth_ext.reindex(columns=meth_int.columns)

# Convert to numpy arrays
meth_int_np = meth_int.values
meth_ext_np = meth_ext.values

# Compute column means from internal methylation data
mean_values = np.nanmean(meth_int_np, axis=0)

# Fill NaNs
meth_int_np = np.where(np.isnan(meth_int_np), mean_values, meth_int_np)
meth_ext_np = np.where(np.isnan(meth_ext_np), mean_values, meth_ext_np)

# Back to DataFrames
meth_int = pd.DataFrame(meth_int_np, index=meth_int.index, columns=meth_int.columns)
meth_ext = pd.DataFrame(meth_ext_np, index=meth_ext.index, columns=meth_ext.columns)


# ========== Standardize Methylation Data ==========

# Fit scaler on internal (training) data only
scaler = StandardScaler()
meth_int_scaled = scaler.fit_transform(meth_int)

# Apply the same scaler to external (test) data
meth_ext_scaled = scaler.transform(meth_ext)


# ========== PCA ==========
pca = PCA(n_components=400)
meth_int_pca = pd.DataFrame(pca.fit_transform(meth_int_scaled), index=meth_int.index)
meth_ext_pca = pd.DataFrame(pca.transform(meth_ext_scaled), index=meth_ext.index)

# Convert all column names to strings to avoid mixed-type errors during model training
meth_int_pca.columns = [f"PCA_{i}" for i in range(meth_int_pca.shape[1])]
meth_ext_pca.columns = meth_int_pca.columns.copy()

# Concatenate meta-features (clinical + methylation + first-layer predictions)
X_train = pd.concat([cli_int, meth_int_pca, pred_int_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
X_test = pd.concat([cli_ext, meth_ext_pca, pred_ext_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

y_train = age_int
y_test = age_ext

# convert object columns to category to avoid error
X_train = convert_object_to_category(X_train)
X_test = convert_object_to_category(X_test)

# Train second-layer TabPFN model
model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
model.fit(X_train, y_train)

# Predict on external data
y_pred = model.predict(X_test)

# Save results
output_df = pred_ext_df.copy()
output_df["ensemble_tabpfn"] = y_pred
output_df["y_true"] = y_test
output_df.to_csv(output_path)

# Evaluate performance
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nExternal EPICAGE Fusion Clock Layer2 Results")
print(f"RMSE: {rmse:.2f}")
print(f"MAE : {mae:.2f}")
print(f"R²  : {r2:.2f}")


External EPICAGE Fusion Clock Layer2 Results
RMSE: 9.11
MAE : 7.18
R²  : 0.71


# Ablation Study - EPICAGE - External

# TABPFN without feature injection

In [4]:
from tabpfn import TabPFNRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
from sklearn.decomposition import PCA
import torch

# ========== Paths ==========
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

# Data paths
pred_int_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"
pred_ext_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_tabpfn_boruta_cli_meth_comb_predictions.csv"
meth_int_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
cli_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
meth_ext_path = PROJECT_ROOT / "data_preprocessing" / "outside_deduplicated_methylation_data.parquet"
cli_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_raw clinical data.csv"
age_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_age.csv"
output_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_no_pca_tabpfn_feature_injection_predictions.csv"

# Load data
#meth_int = pd.read_parquet(meth_int_path)
#cli_int = pd.read_csv(cli_int_path, index_col=0)
age_int = pd.read_csv(age_int_path).set_index("attrib_name")["years_to_birth"]
#meth_ext = pd.read_parquet(meth_ext_path)
#cli_ext = pd.read_csv(cli_ext_path, index_col=0)
age_ext = pd.read_csv(age_ext_path).set_index("attrib_name")["years_to_birth"]

pred_int_df = pd.read_csv(pred_int_path, index_col=0)
pred_ext_df = pd.read_csv(pred_ext_path, index_col=0)

'''
# ========== Fill missing CpGs ==========

# Fill NaNs in meth_int with column-wise means
meth_int_array = meth_int.values
col_means_int = np.nanmean(meth_int_array, axis=0)
nan_mask_int = np.isnan(meth_int_array)
rows_int, cols_int = np.where(nan_mask_int)
meth_int_array[rows_int, cols_int] = col_means_int[cols_int]

meth_int = pd.DataFrame(meth_int_array, index=meth_int.index, columns=meth_int.columns).copy()

# Get all required columns from internal data
all_columns = meth_int.columns

# Find missing columns in external methylation
missing_cols = list(set(all_columns) - set(meth_ext.columns))

# Fill missing columns with the global mean of meth_ext
if missing_cols:
    global_ext_mean = np.nanmean(meth_ext.values)
    fill_df = pd.DataFrame(global_ext_mean, index=meth_ext.index, columns=missing_cols)
    meth_ext = pd.concat([meth_ext, fill_df], axis=1)

# Reorder columns to match internal structure
meth_ext = meth_ext[all_columns]

# Vectorized fill of NaNs in existing columns with per-column means
meth_ext_array = meth_ext.values
col_means = np.nanmean(meth_ext_array, axis=0)
nan_mask = np.isnan(meth_ext_array)
rows, cols = np.where(nan_mask)
meth_ext_array[rows, cols] = col_means[cols]

# Assign back to DataFrame to avoid fragmentation
meth_ext = pd.DataFrame(meth_ext_array, index=meth_ext.index, columns=meth_ext.columns).copy()

# Fill NaNs in meth_int with column-wise means
meth_int_array = meth_int.values
col_means_int = np.nanmean(meth_int_array, axis=0)
nan_mask_int = np.isnan(meth_int_array)
rows_int, cols_int = np.where(nan_mask_int)
meth_int_array[rows_int, cols_int] = col_means_int[cols_int]

meth_int = pd.DataFrame(meth_int_array, index=meth_int.index, columns=meth_int.columns).copy()


# ========== PCA ==========
pca = PCA(n_components=400)
meth_int_pca = pd.DataFrame(pca.fit_transform(meth_int), index=meth_int.index)
meth_ext_pca = pd.DataFrame(pca.transform(meth_ext), index=meth_ext.index)

# Concatenate meta-features (clinical + methylation + first-layer predictions)

X_train = pd.concat([cli_int, meth_int_pca, pred_int_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
X_test = pd.concat([cli_ext, meth_ext_pca, pred_ext_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
'''
X_train = pred_int_df[['pred_cli', 'pred_meth', 'pred_comb']]
X_test = pred_ext_df[['pred_cli', 'pred_meth', 'pred_comb']]

X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

y_train = age_int
y_test = age_ext

# Train second-layer TabPFN model
model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
model.fit(X_train.values, y_train.values)

# Predict on external data
y_pred = model.predict(X_test.values)

# Save results
output_df = pred_ext_df.copy()
output_df["ensemble_tabpfn"] = y_pred
output_df["y_true"] = y_test
output_df.to_csv(output_path)

# Evaluate performance
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nExternal EPICAGE Fusion Clock Layer2 Results (No Feature Injection) ")
print(f"RMSE: {rmse:.2f}")
print(f"MAE : {mae:.2f}")
print(f"R²  : {r2:.2f}")


External EPICAGE Fusion Clock Layer2 Results (No Feature Injection) 
RMSE: 16.85
MAE : 13.52
R²  : 0.00


# External Tabpfn Feature Selection（Use sperman+Boruta to substitute PCA）

In [5]:
from tabpfn import TabPFNRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
import torch

# ========== Paths ==========
PROJECT_ROOT = Path("/content/drive/My Drive/Age_Prediction")

pred_int_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "tabpfn_boruta_cli_meth_comb_predictions.csv"
pred_ext_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_tabpfn_boruta_cli_meth_comb_predictions.csv"
meth_int_path = PROJECT_ROOT / "data_preprocessing" / "deduplicated_methylation_data.parquet"
cli_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "raw clinical data.csv"
age_int_path = PROJECT_ROOT / "data_merge" / "Raw Data" / "age.csv"
meth_ext_path = PROJECT_ROOT / "data_preprocessing" / "outside_deduplicated_methylation_data.parquet"
cli_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_raw clinical data.csv"
age_ext_path = PROJECT_ROOT / "data_merge" / "Outside Raw Data" / "outside_age.csv"
boruta_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "Boruta" / "feature selection" / "borutashap_all_internal_features.csv"
output_path = PROJECT_ROOT / "Our model" / "EPICAGE" / "results" / "external_boruta_tabpfn_feature_injection_predictions.csv"

# ========== Load Data ==========
meth_int = pd.read_parquet(meth_int_path)
cli_int = pd.read_csv(cli_int_path, index_col=0)
age_int = pd.read_csv(age_int_path).set_index("attrib_name")["years_to_birth"]
meth_ext = pd.read_parquet(meth_ext_path)
cli_ext = pd.read_csv(cli_ext_path, index_col=0)
age_ext = pd.read_csv(age_ext_path).set_index("attrib_name")["years_to_birth"]
pred_int_df = pd.read_csv(pred_int_path, index_col=0)
pred_ext_df = pd.read_csv(pred_ext_path, index_col=0)

# ========== Align Sample IDs ==========
common_int_ids = meth_int.index.intersection(cli_int.index).intersection(age_int.index).intersection(pred_int_df.index)
meth_int = meth_int.loc[common_int_ids]
cli_int = cli_int.loc[common_int_ids]
age_int = age_int.loc[common_int_ids]
pred_int_df = pred_int_df.loc[common_int_ids]

common_ext_ids = meth_ext.index.intersection(cli_ext.index).intersection(age_ext.index).intersection(pred_ext_df.index)
meth_ext = meth_ext.loc[common_ext_ids]
cli_ext = cli_ext.loc[common_ext_ids]
age_ext = age_ext.loc[common_ext_ids]
pred_ext_df = pred_ext_df.loc[common_ext_ids]

# ========== Load Boruta CpGs ==========
selected_cpgs = pd.read_csv(boruta_path, header=None, skiprows=1).squeeze("columns").tolist()



# ----------- ---------Align external data to internal data----------------------------------
# Ensure all required CpGs exist in external data
all_columns = meth_int.columns
missing_cols = list(set(all_columns) - set(meth_ext.columns))
if missing_cols:
    global_mean_ext = np.nanmean(meth_ext.values)
    for cpg in missing_cols:
        meth_ext[cpg] = global_mean_ext

# Reorder and impute external methylation
meth_ext = meth_ext[all_columns]
meth_ext_array = meth_ext.values
col_means_ext = np.nanmean(meth_ext_array, axis=0)
meth_ext_array[np.isnan(meth_ext_array)] = np.take(col_means_ext, np.where(np.isnan(meth_ext_array))[1])
meth_ext = pd.DataFrame(meth_ext_array, index=meth_ext.index, columns=meth_ext.columns)



# Ensure consistent column order
meth_int = meth_int[selected_cpgs]
meth_ext = meth_ext[selected_cpgs]

# ========== Fill NaNs with Column Means ==========
meth_int = meth_int.fillna(meth_int.mean())
meth_ext = meth_ext.fillna(meth_ext.mean())

# ========== Feature Injection ==========
X_train = pd.concat([cli_int, meth_int, pred_int_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
X_test = pd.concat([cli_ext, meth_ext, pred_ext_df[['pred_cli', 'pred_meth', 'pred_comb']]], axis=1)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

y_train = age_int
y_test = age_ext

# convert object columns to category to avoid error
X_train = convert_object_to_category(X_train)
X_test = convert_object_to_category(X_test)

# ========== Train Ensemble Model ==========
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TabPFNRegressor(device=device)
model.fit(X_train, y_train)

# ========== Predict on External Data ==========
y_pred = model.predict(X_test)

# ========== Save Predictions ==========
output_df = pred_ext_df.copy()
output_df["ensemble_tabpfn"] = y_pred
output_df["y_true"] = y_test
output_df.to_csv(output_path)

# ========== Evaluate Performance ==========
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nExternal EPICAGE Fusion Clock Layer2 Results (Sperman + Boruta Feature Injection)")
print(f"RMSE: {rmse:.2f}")
print(f"MAE : {mae:.2f}")
print(f"R²  : {r2:.2f}")

/tmp/ipython-input-3149787831.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meth_ext[cpg] = global_mean_ext
/tmp/ipython-input-3149787831.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meth_ext[cpg] = global_mean_ext
/tmp/ipython-input-3149787831.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fram


External EPICAGE Fusion Clock Layer2 Results (Sperman + Boruta Feature Injection)
RMSE: 9.43
MAE : 7.34
R²  : 0.69
